In [ ]:
import requests
import json
import time
from datetime import datetime
import psycopg2
from psycopg2 import sql

# Deine API-Key von OpenWeatherMap
API_KEY = "bd5e378503939ddaee76f12ad7a97608"

# IDs der Städte, die abgefragt werden sollen
city_ids = [
    2761369,  # Wien
    2772400,  # Linz
    2778067,  # Graz
    2766824,  # Salzburg
    2775220,  # Innsbruck
    7871497,  # Klagenfurt
    2781503,  # Bregenz
    2782045   # Bad Tatzmannsdorf
]

# URL für den API-Call
def get_weather_data(ids_param):
    url = (
        f"https://api.openweathermap.org/data/2.5/group"
        f"?id={ids_param}"
        f"&units=metric"
        f"&appid={API_KEY}"
    )
    response = requests.get(url)
    return response.json()

# Funktion, um die Daten zu speichern (auf Datei)
def save_data(data):
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    with open(f"weather_data_{timestamp}.json", 'w') as f:
        json.dump(data, f, indent=4)

# Funktion, um die Daten auszugeben
def print_weather_data(weather_data):
    for city_data in weather_data['list']:
        print(f"\nStadt: {city_data['name']} ({city_data['id']})")
        print(f"  - Temperatur: {city_data['main']['temp']}°C")
        print(f"  - Min. Temp: {city_data['main']['temp_min']}°C")
        print(f"  - Max. Temp: {city_data['main']['temp_max']}°C")
        print(f"  - Luftdruck: {city_data['main']['pressure']} hPa")
        print(f"  - Luftfeuchtigkeit: {city_data['main']['humidity']}%")
        print(f"  - Windgeschwindigkeit: {city_data['wind']['speed'] * 3.6:.2f} km/h")  # Umgerechnet in km/h
        print(f"  - Windrichtung: {city_data['wind']['deg']}°")

# Funktion zum Einfügen von Wetterdaten in die DB
def insert_weather_data(cursor, weather_data):
    for city_data in weather_data['list']:
        city_info = {
            "city_id": city_data["id"],
            "city_name": city_data["name"],
            "latitude": city_data["coord"]["lat"],
            "longitude": city_data["coord"]["lon"],
            "timestamp_utc": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'),
            "temp_celsius": city_data["main"]["temp"],
            "temp_min_c": city_data["main"]["temp_min"],
            "temp_max_c": city_data["main"]["temp_max"],
            "pressure_hpa": city_data["main"]["pressure"],
            "humidity_pct": city_data["main"]["humidity"],
            "wind_speed_kph": city_data["wind"]["speed"] * 3.6,  # m/s in km/h umrechnen
            "wind_direction": city_data["wind"]["deg"]
        }

        insert_query = sql.SQL("""
            INSERT INTO weather_data_api (
                city_id, city_name, latitude, longitude, 
                timestamp_utc, temp_celsius, temp_min_c,
                temp_max_c, pressure_hpa, humidity_pct,
                wind_speed_kph, wind_direction
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """)

        cursor.execute(insert_query, (
            city_info["city_id"], city_info["city_name"], city_info["latitude"], city_info["longitude"],
            city_info["timestamp_utc"], city_info["temp_celsius"], city_info["temp_min_c"],
            city_info["temp_max_c"], city_info["pressure_hpa"], city_info["humidity_pct"],
            city_info["wind_speed_kph"], city_info["wind_direction"]
        ))

# Verbindung zur DB herstellen
def get_db_connection():
    conn = psycopg2.connect(
        dbname="weather_db", 
        user="bdeng", 
        password="bdengpass", 
        host="localhost",  # Wenn du den Docker-Container verwendest, kannst du den Container-Namen statt localhost verwenden
        port="5432"
    )
    return conn

# Funktion zum Erstellen der Tabelle, falls sie nicht existiert
def create_table_if_not_exists(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS weather_data_api (
        city_id INT,
        city_name VARCHAR(255),
        latitude FLOAT,
        longitude FLOAT,
        timestamp_utc TIMESTAMP,
        temp_celsius FLOAT,
        temp_min_c FLOAT,
        temp_max_c FLOAT,
        pressure_hpa FLOAT,
        humidity_pct INT,
        wind_speed_kph FLOAT,
        wind_direction INT,
        PRIMARY KEY (city_id, timestamp_utc)  -- Primärschlüssel auf city_id und timestamp_utc setzen
    );
    """
    cursor.execute(create_table_query)

# Hauptfunktion
def main():
    count = 0
    all_data = []  # Liste, um alle Abfragen zu speichern

    # IDs als String für den API-Call zusammenfügen
    ids_param = ",".join(map(str, city_ids))

    # Verbindung zur DB herstellen
    conn = get_db_connection()
    cursor = conn.cursor()

    # Tabelle erstellen, falls sie noch nicht existiert
    create_table_if_not_exists(cursor)

    while count < 1000:
        # Wetterdaten abfragen
        weather_data = get_weather_data(ids_param)

        # Hinzufügen des Zeitstempels und Verarbeiten der Daten
        timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        for city_data in weather_data['list']:
            city_info = {
                "city_id": city_data["id"],
                "city_name": city_data["name"],
                "latitude": city_data["coord"]["lat"],
                "longitude": city_data["coord"]["lon"],
                "timestamp_utc": timestamp,
                "temp_celsius": city_data["main"]["temp"],
                "temp_min_c": city_data["main"]["temp_min"],
                "temp_max_c": city_data["main"]["temp_max"],
                "pressure_hpa": city_data["main"]["pressure"],
                "humidity_pct": city_data["main"]["humidity"],
                "wind_speed_kph": city_data["wind"]["speed"] * 3.6,  # m/s in km/h umrechnen
                "wind_direction": city_data["wind"]["deg"]
            }
            all_data.append(city_info)

        # Ausgabe der Daten
        print(f"\nAbfrage {count + 1} von 1000:")
        print_weather_data(weather_data)

        # Wetterdaten in die DB einfügen
        insert_weather_data(cursor, weather_data)

        # Änderungen in der DB speichern
        conn.commit()

        # Speichern der gesammelten Daten alle 10 Abfragen
        if (count + 1) % 10 == 0:
            save_data(all_data)
            print(f"{count + 1} Abfragen abgeschlossen, Daten gespeichert.")

        # Pause von 10 Sekunden
        time.sleep(10)
        count += 1

    # Endgültige Speicherung nach 1000 Abfragen
    save_data(all_data)
    print(f"1000 Abfragen abgeschlossen, finale Daten gespeichert.")

    # DB-Verbindung schließen
    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()

C:\Users\jvanb\AppData\Local\Temp\ipykernel_30328\408533309.py:139: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
C:\Users\jvanb\AppData\Local\Temp\ipykernel_30328\408533309.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_utc": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'),



Abfrage 1 von 1000:

Stadt: Vienna (2761369)
  - Temperatur: 22.89°C
  - Min. Temp: 21.75°C
  - Max. Temp: 23.49°C
  - Luftdruck: 1022 hPa
  - Luftfeuchtigkeit: 38%
  - Windgeschwindigkeit: 27.36 km/h
  - Windrichtung: 65°

Stadt: Linz (2772400)
  - Temperatur: 24.24°C
  - Min. Temp: 24.24°C
  - Max. Temp: 24.24°C
  - Luftdruck: 1023 hPa
  - Luftfeuchtigkeit: 38%
  - Windgeschwindigkeit: 11.12 km/h
  - Windrichtung: 60°

Stadt: Graz (2778067)
  - Temperatur: 23.12°C
  - Min. Temp: 22.81°C
  - Max. Temp: 23.29°C
  - Luftdruck: 1022 hPa
  - Luftfeuchtigkeit: 52%
  - Windgeschwindigkeit: 9.25 km/h
  - Windrichtung: 80°

Stadt: Salzburg (2766824)
  - Temperatur: 24.95°C
  - Min. Temp: 24.95°C
  - Max. Temp: 24.95°C
  - Luftdruck: 1023 hPa
  - Luftfeuchtigkeit: 41%
  - Windgeschwindigkeit: 11.12 km/h
  - Windrichtung: 10°

Stadt: Innsbruck (2775220)
  - Temperatur: 25.93°C
  - Min. Temp: 23.49°C
  - Max. Temp: 26.03°C
  - Luftdruck: 1022 hPa
  - Luftfeuchtigkeit: 44%
  - Windgeschwindigkei